In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
#from sparknlp.base import Finisher, DocumentAssembler
#from sparknlp.annotator import  Normalizer, LemmatizerModel, StopWordsCleaner, Tokenizer
#from pyspark.ml import Pipeline
#from pyspark.ml.feature import CountVectorizer, VectorAssembler, HashingTF, IDF, Tokenizer
#from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, plot_confusion_matrix, \
                            roc_curve, auc, classification_report, precision_recall_curve
import matplotlib.pyplot as plt
#RegEx
import re

import scrubadub
#import scrubadub[address]




VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1608046971644_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  r.review_headline, r.review_comments, r.reviewer_nickname, r.reviewer_location, \
case when (contains(r.observations, 'PII') )  then 1 else 0 end as PII \
from analytics.review as r \
where r.locale in ('en_US','en_GB','en_CA') and contains(r.observations, 'PM') \
and r.created_date > '2020-07-01' "


#and contains(r.observations, 'PII') \
#and ugc_id in (270519748, 202761051, 202759202, 203453834, 135796841, 135854691, 217517666, 275335901, 264483850, 175108572, 175148446) \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head(20)
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID  ... PII
0   319242467  ...   0
1   318133873  ...   0
2   318136461  ...   0
3   318138108  ...   0
4   318143623  ...   0
5   318146407  ...   0
6   318148490  ...   0
7   318148441  ...   0
8   318155843  ...   0
9   319290290  ...   0
10  319294875  ...   0
11  319241987  ...   0
12  318151255  ...   0
13  319318687  ...   0
14  318192451  ...   0
15  318193676  ...   0
16  318193258  ...   0
17  318192285  ...   0
18  318193276  ...   0
19  318192203  ...   0

[20 rows x 6 columns]

In [ ]:
reviews.info()

In [6]:
#Force space at the end so that regex will match space after location
#This is necessary so if they don't use a zip code and the address ends in just the state abbreviation, it will pick it up 
#But will not pick up in text where something is a word that starts with a state's 2 letter abbreviation and thus creates lots of false positives

reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS'] + ' '+reviews['REVIEWER_NICKNAME'] + ' '+reviews['REVIEWER_LOCATION'] +' '
#Use set_option so text doesn't truncate

#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
reviews.head()

#reviews.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ...                                           ALL_TEXT
0  319242467  ...  Good product I bought these a week ago and I l...
1  318133873  ...  Sale sale n more sale Sale sale n more sale oe...
2  318136461  ...                  INCREDIBLE INCREDIBLE aqy9716 US 
3  318138108  ...  Great love the makeup Great love the makeup eg...
4  318143623  ...  Your website doesn't allow you to go back to t...

[5 rows x 7 columns]

In [7]:
reviews['regex_creditcard'] = reviews['ALL_TEXT'].str.contains('\d{4}(\s|-|\.|\*)\d{4}(\s|-|\.|\*)\d{4}(\s|-|\.|\*)\d{4}')== True

reviews['regex_socsec']  = reviews['ALL_TEXT'].str.contains('\d{3}(\s|-|\.|\*)\d{2}(\s|-|\.|\*)\d{4}')== True

reviews['regex_phone10'] = reviews['ALL_TEXT'].str.contains('(\d(\s|-|\.|\*)?)?(\(?\d{3}\)?|\d{3})(\s|-|\.|\*)\d{3}(\s|-|\.|\*)\d{4}')== True

reviews['regex_email'] = reviews['ALL_TEXT'].str.contains('[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}')== True


#Handle regex will pick up majority of email addresses
reviews['regex_handle'] = reviews['ALL_TEXT'].str.contains('@[A-Za-z0-9\.\_]+')== True

#MUST MAKE CASE INSENSITIVE
#Apply to each text field separately bc otherwise if apply to ALL_TEXT then can match # from comments and word from nickname
#and city, state from location and get a match
reviews['regex_USaddress_loc'] = reviews['REVIEWER_LOCATION']. \
str.contains(r'\d{1,6}(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)\s(STREET|ST|DRIVE|DR|AVENUE|AVE|ROAD|RD|LOOP|COURT|CT|CIRCLE|LANE|LN|BOULEVARD|BLVD|COVE|TERRACE|TERR|TER)\.?\,?(\s(APARTMENT|APT|BUILDING|BLDG|DEPARTMENT|DEPT|FLOOR|FL|LOT|PIER|ROOM|RM|TRAILER|TRLR|UNIT|PO BOX)\.?\s\#?[A-Za-z0-9]+\,?)?(\s[A-Za-z]+)?(\s[A-Za-z]+)\,?\s(A[LKSZR]|C[AOT]|D[EC]|F[L]|G[AU]|HI|I[ADLN]|K[SY]|LA|M[ADEINOPST]|N[CDEHJMVY]|O[HKR]|P[AR]|RI|S[CD]|T[NX]|UT|V[AIT]|W[AIVY])(\s\d{5})?(\-\d{4})?', case=False)== True

#Require word boundary after state code in the comments field since such long text, not necessary in location or headline field
#Otherwise will match "50 mile drive one way...." bc word "way" contains WA as in Washington state
#HOWEVER then we are losing if it was legit but they decided to type out Washington instead of just WA
reviews['regex_USaddress_comments'] = reviews['REVIEW_COMMENTS']. \
str.contains(r'\d{1,6}(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)\s(STREET|ST|DRIVE|DR|AVENUE|AVE|ROAD|RD|LOOP|COURT|CT|CIRCLE|LANE|LN|BOULEVARD|BLVD|COVE|TERRACE|TERR|TER)\.?\,?(\s(APARTMENT|APT|BUILDING|BLDG|DEPARTMENT|DEPT|FLOOR|FL|LOT|PIER|ROOM|RM|TRAILER|TRLR|UNIT|PO BOX)\.?\s\#?[A-Za-z0-9]+\,?)?(\s[A-Za-z]+)?(\s[A-Za-z]+)\,?\s(A[LKSZR]|C[AOT]|D[EC]|F[L]|G[AU]|HI|I[ADLN]|K[SY]|LA|M[ADEINOPST]|N[CDEHJMVY]|O[HKR]|P[AR]|RI|S[CD]|T[NX]|UT|V[AIT]|W[AIVY])\b(\s\d{5})?(\-\d{4})?', case=False)== True

reviews['regex_USaddress_headline'] = reviews['REVIEW_HEADLINE']. \
str.contains(r'\d{1,6}(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)?(\s[A-Za-z0-9]+\.?\,?)\s(STREET|ST|DRIVE|DR|AVENUE|AVE|ROAD|RD|LOOP|COURT|CT|CIRCLE|LANE|LN|BOULEVARD|BLVD|COVE|TERRACE|TERR|TER)\.?\,?(\s(APARTMENT|APT|BUILDING|BLDG|DEPARTMENT|DEPT|FLOOR|FL|LOT|PIER|ROOM|RM|TRAILER|TRLR|UNIT|PO BOX)\.?\s\#?[A-Za-z0-9]+\,?)?(\s[A-Za-z]+)?(\s[A-Za-z]+)\,?\s(A[LKSZR]|C[AOT]|D[EC]|F[L]|G[AU]|HI|I[ADLN]|K[SY]|LA|M[ADEINOPST]|N[CDEHJMVY]|O[HKR]|P[AR]|RI|S[CD]|T[NX]|UT|V[AIT]|W[AIVY])(\s\d{5})?(\-\d{4})?', case=False)== True


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib64/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/usr/local/lib64/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [8]:
reviews['regex_PII'] = 0
reviews.loc[(reviews['regex_creditcard']==True) | (reviews['regex_socsec']==True) | (reviews['regex_phone10']==True) \
         | (reviews['regex_email']==True) | (reviews['regex_handle']==True) | (reviews['regex_USaddress_loc']==True) \
        | (reviews['regex_USaddress_comments']==True) | (reviews['regex_USaddress_headline']==True), 'regex_PII'] = 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ... regex_PII
0  319242467  ...         0
1  318133873  ...         0
2  318136461  ...         0
3  318138108  ...         0
4  318143623  ...         0

[5 rows x 16 columns]

In [10]:
FP = reviews[(reviews['PII'] == 0) & (reviews['regex_PII'] == 1)]
FN = reviews[(reviews['PII'] == 1) & (reviews['regex_PII'] == 0)]
TP = reviews[(reviews['PII'] == 1) & (reviews['regex_PII'] == 1)]
TN = reviews[(reviews['PII'] == 0) & (reviews['regex_PII'] == 0)]
FP['UGC_ID'].count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8478

In [11]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

61750

In [12]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4552

In [13]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7943445

In [14]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

test = FP[['ALL_TEXT','PII','regex_PII']]
test.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("REVIEWER_NICKNAME", StringType(), True)\
,StructField("REVIEWER_LOCATION", StringType(), True)\
,StructField("PII", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("regex_creditcard", BooleanType(), True)\
,StructField("regex_soc_sec", BooleanType(), True)\
,StructField("regex_phone10", BooleanType(), True)\
,StructField("regex_email", BooleanType(), True)\
,StructField("regex_handle", BooleanType(), True)\
,StructField("regex_USaddress_loc", BooleanType(), True)\
,StructField("regex_USaddress_comments", BooleanType(), True)\
,StructField("regex_USaddress_headline", BooleanType(), True)\
,StructField("regex_PII", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
reviews = spark.createDataFrame(reviews, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://10.101.213.215:18888/sessions/6/statements/16 with error payload: {"msg":"requirement failed: Session isn't active."}


In [17]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

An error was encountered:
Session 6 unexpectedly reached final status 'dead'. See logs:
stdout: 
[ WARN] Your hostname, ip-10-101-213-215.us-west-2.compute.internal, resolves to a loopback address, but we couldn't find any external IP address!
[ WARN] Set livy.rsc.rpc.server.address if you need to bind to another address.
[ WARN] Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
[ WARN] Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
[ WARN] Same name resource file:///usr/lib/spark/python/lib/pyspark.zip added multiple times to distributed cache
[ WARN] Same name resource file:///usr/lib/spark/python/lib/py4j-0.10.7-src.zip added multiple times to distributed cache
#
# java.lang.OutOfMemoryError: Java heap space
# -XX:OnOutOfMemoryError="kill -9 %p"
#   Executing /bin/sh -c "kill -9 2037"...

stderr: 

YARN Diagnostics: 


In [18]:
reviews.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","PII_test1").mode("append").save()


VBox()

An error was encountered:
Session 6 unexpectedly reached final status 'dead'. See logs:
stdout: 
[ WARN] Your hostname, ip-10-101-213-215.us-west-2.compute.internal, resolves to a loopback address, but we couldn't find any external IP address!
[ WARN] Set livy.rsc.rpc.server.address if you need to bind to another address.
[ WARN] Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
[ WARN] Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
[ WARN] Same name resource file:///usr/lib/spark/python/lib/pyspark.zip added multiple times to distributed cache
[ WARN] Same name resource file:///usr/lib/spark/python/lib/py4j-0.10.7-src.zip added multiple times to distributed cache
#
# java.lang.OutOfMemoryError: Java heap space
# -XX:OnOutOfMemoryError="kill -9 %p"
#   Executing /bin/sh -c "kill -9 2037"...

stderr: 

YARN Diagnostics: 


In [8]:
#Ok if it doesn't get phone # with 7 digits bc no one could call without area code anyway
#Would mark any location with Name, only want addresses with street info in them
text = "My name is Apple Smith can be contacted on example@example.com, or 1800 555-5555 or 123-3421 or +1(800)452-1234 \
soc sec num 123-02-3240 or without dashes 123023240 or with periods 123.02.3240 \
or at 414 Wrightwood Ave Chicago, IL or in Chicago or in Chicago, IL. \
I work at The North Face. Is this a Name? \
Credit card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. \
Website is http://www.amazon.com or www.amazon.com or amazon.com"
 

# Replaces the phone number and email addresse with anonymous IDs.
scrubadub.clean(text)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'My name is {{NAME}} {{NAME}} can be contacted on {{EMAIL}}, or {{PHONE}} or 123-3421 or {{PHONE}} soc sec num {{SSN}} or without dashes 123023240 or with periods {{SSN}} or at 414 {{NAME}} {{NAME}} {{NAME}}, IL or in {{NAME}} or in {{NAME}}, IL. I work at The {{NAME}} {{NAME}}. Is this a Name? {{NAME}} card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. {{NAME}} is {{URL}} or {{URL}} or amazon.com'

In [9]:
scrubber = scrubadub.Scrubber()
scrubber.remove_detector('name')
scrubber.clean(text)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'My name is Apple Smith can be contacted on {{EMAIL}}, or {{PHONE}} or 123-3421 or {{PHONE}} soc sec num {{SSN}} or without dashes 123023240 or with periods {{SSN}} or at 414 Wrightwood Ave Chicago, IL or in Chicago or in Chicago, IL. I work at The North Face. Is this a Name? Credit card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. Website is {{URL}} or {{URL}} or amazon.com'

In [14]:
txt = "My credit card is 1234 123412341234"
x = re.search('\d{4}(-|\.|\s)?\d{4}(-|\.|\s)?\d{4}(-|\.|\s)?\d{4}', txt)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
x

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<re.Match object; span=(18, 35), match='1234 123412341234'>